In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, gc, random
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa
from tf_nn_utils import *
from viterbi_utils_sandbox import *
from sklearn.preprocessing import MinMaxScaler

import numpy.fft as fft
from scipy import signal as scisig
from viterbi_utils import *
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

# set gpu memory growth
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(42)

In [2]:
df_train = pd.read_pickle('../features/train_clean.pkl')
df_test = pd.read_pickle('../features/test_clean.pkl')
TARGET = "open_channels"
df_test[TARGET] = 0

df_train["group"] = df_train["batch"].astype("str") + "_" + df_train["mini_batch"].astype("str")
df_test["group"] = df_test["batch"].astype("str") + "_" + df_test["mini_batch"].astype("str")

df_train["signal_original"] = df_train["signal"].copy()
df_test["signal_original"] = df_test["signal"].copy()

print(f"train size:{df_train.shape}, test size:{df_test.shape}")
df_train.head()

train size:(4500000, 8), test size:(2000000, 9)


,time,signal,open_channels,local_time,batch,mini_batch,group,signal_original
0,0.0001,-2.7600,0,0.0001,1,1,1_1,-2.7600
1,0.0002,-2.8557,0,0.0002,1,1,1_1,-2.8557
2,0.0003,-2.4074,0,0.0003,1,1,1_1,-2.4074
3,0.0004,-3.1404,0,0.0004,1,1,1_1,-3.1404
4,0.0005,-3.1525,0,0.0005,1,1,1_1,-3.1525


In [3]:
# configurations and main hyperparammeters
shft = 1
nn_epochs = 100
nn_batch_size = 16
class_num = 11 - shft
LR = 0.001

In [4]:
# reduce batch 5&10 open_channels from 11 to 10 (class 1-10)
df_train.loc[df_train.batch.isin([5,10]) & (df_train.open_channels < shft), "open_channels"] = shft
df_train[TARGET] = df_train[TARGET] - shft

# mini model
BATCH_GROUP = [5, 10]
df_train = df_train[df_train.batch.isin(BATCH_GROUP)].reset_index(drop=True)
TEST_GROUP = ["2_1", "2_3"]
df_test = df_test[df_test.group.isin(TEST_GROUP)].reset_index(drop=True)

print(f"train size:{df_train.shape}, test size:{df_test.shape}")

train size:(1000000, 8), test size:(200000, 9)


In [5]:
# # remove the 50 hz noise using bandstop filter (group)
# for group_i in df_train.group.unique():

#     batch_i = df_train[df_train.group.isin([group_i])]
#     signal_recovered = rm_noise(batch_i)
#     df_train.loc[df_train.group.isin([group_i]), "signal"] = signal_recovered

In [6]:
sig_mean = get_mean(df_train)
print(np.array(sig_mean).round(3))
# sig_mean = [-4.255, -3.017, -1.779, -0.541, 0.697, 1.935, 3.173, 4.411, 5.649, 6.887]

[-4.218 -2.969 -1.768 -0.526  0.71   1.941  3.175  4.409  5.641  6.875]


In [7]:
# remove the 50 hz noise using bandstop filter (batch)
for batch_idx in df_train.batch.unique():

    batch_i = df_train[df_train.batch.isin([batch_idx])]
    signal_recovered = rm_noise(batch_i, sig_mean=sig_mean)
    df_train.loc[df_train.batch.isin([batch_idx]), "signal"] = signal_recovered

In [8]:
sig_mean = get_mean(df_train)
print(np.array(sig_mean).round(3))
# sig_mean = [-4.255, -3.017, -1.779, -0.541, 0.697, 1.935, 3.173, 4.411, 5.649, 6.887]

[-4.212 -2.959 -1.766 -0.523  0.71   1.941  3.175  4.409  5.641  6.875]


In [9]:
# feature engineering here
def fe(df, is_train):

    # shift features
    for shift_val in range(1, 2):
        group_on = "group"
        df['shift'] = df.groupby([group_on])['signal'].shift(shift_val).fillna(0)
    
    return df

df_train = fe(df_train, is_train=1)
df_test = fe(df_test, is_train=0)

use_cols = [
    col for col in df_train.columns if col not in
    ["time", "local_time", "open_channels", "batch", "mini_batch", "group", "oof", "signal_original"]
]
print("Used columns is", use_cols)

Used columns is ['signal', 'shift']


In [10]:
viterbi_cols = ["viterbi_" + str(i) for i in range(df_train[TARGET].nunique())]
use_cols = use_cols + viterbi_cols

for col in viterbi_cols:
    df_train[col] = 0
    df_test[col] = 0
    
# see performance in each group
cols = ["signal", "shift"]
signals = df_train.loc[df_train.batch.isin(BATCH_GROUP), cols].values#.reshape([-1, 1])
state = df_train.loc[df_train.batch.isin(BATCH_GROUP), TARGET].values
init_prob = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
ghmm = GaussHMM(init_prob)
# ghmm.fit(signals, state)


In [11]:
def get_cov(signals, channels):

    sig_cov = []
    for chan_i in range(len(np.unique(channels))):
        sig_cov.append(np.cov(signals[channels == chan_i].reshape([-1, 2]).T))

    return np.array(sig_cov).reshape([-1, 2, 2])

In [12]:
def get_mean(signals, channels):

    sig_mean = []
    for chan_i in range(len(np.unique(channels))):
        sig_mean.append(signals[channels == chan_i].mean())

    return np.array(sig_mean).reshape([-1, 2])

In [13]:
def markov_p_trans(states):
    max_state = np.max(states)
    states_next = np.roll(states, -1)
    matrix = []
    for i in range(max_state + 1):
        current_row = np.histogram(states_next[states == i], bins=np.arange(max_state + 2))[0]
        if np.sum(current_row) == 0: # if a state doesn't appear in states...
            current_row = np.ones(max_state + 1) / (max_state + 1) # ...use uniform probability
        else:
            current_row = current_row / np.sum(current_row) # normalize to 1
        matrix.append(current_row)
    return np.array(matrix)

In [14]:
hmm = GaussianHMM(n_components=len(init_prob), covariance_type="full", n_iter=100)
hmm.fit(np.array(signals).reshape([-1, 2])[:200])
hmm.means_ = get_mean(signals, state)
hmm.covars_ = get_cov(signals, state)
hmm.startprob_ = init_prob
hmm.transmat_ = markov_p_trans(state)

In [17]:

for group_i in df_train.group.unique():
    batch_i = df_train[df_train.group.isin([group_i])]
    signal_i = df_train.loc[df_train.group.isin([group_i]), cols].values
    state_i = df_train.loc[df_train.group.isin([group_i]), TARGET].values
    df_train.loc[df_train.group.isin([group_i]), "oof"] = hmm.predict(signal_i)
#     df_train.loc[df_train.group.isin([group_i]), viterbi_cols] = hmm.predict_proba(signal_i)
    print(group_i, "F1 macro =", f1_score(y_pred=df_train.loc[df_train.group.isin([group_i]), "oof"], y_true=state_i, average='macro').round(6))

print("==> OOF F1 macro =", f1_score(y_pred=df_train["oof"].values, y_true=df_train[TARGET].values, average='macro').round(4))

5_1 F1 macro = 2.4e-05
5_2 F1 macro = 8e-06
5_3 F1 macro = 1e-05
5_4 F1 macro = 5.4e-05
5_5 F1 macro = 2.2e-05
10_1 F1 macro = 2e-05
10_2 F1 macro = 2.6e-05
10_3 F1 macro = 2.8e-05
10_4 F1 macro = 2.2e-05
10_5 F1 macro = 4.2e-05
==> OOF F1 macro = 0.0


In [18]:
df_train["oof"]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
999995    0.0
999996    0.0
999997    0.0
999998    0.0
999999    0.0
Name: oof, Length: 1000000, dtype: float64

In [ ]:
stop

In [ ]:
def get_noise(batch, sig_mean, col="open_channels"):
    """
    input: batch df
    output: recovered signal
    """
    signal = batch.signal_original.values
    channels = batch[col].values
    # sig_mean = get_mean(batch, col=col)
    sig_noise = Arrange_mean(signal, channels, sig_mean, len(sig_mean))

    return sig_noise

In [ ]:
# predict the test
df_test["noise"] = 0
for col in TEST_GROUP:
    df_test.loc[df_test.group == col, "oof"] = ghmm.predict(df_test.loc[df_test.group == col, "signal"].values)
    df_test.loc[df_test.group == col, viterbi_cols] = ghmm.predict_proba(df_test.loc[df_test.group == col, "signal"].values)
    df_test.loc[df_test.group == col, "noise"] = get_noise(df_test[df_test.group == col], sig_mean, col="oof")
    
df_test["max_typicality"] = df_test[viterbi_cols].values.max(axis=1)

# for i in range(5):
#     # remove test signal noise here using oof
#     for group_i in TEST_GROUP:
#         batch_test_i = df_test[df_test.group.isin([group_i])].copy()
#         signal_test_recovered = rm_noise(batch_test_i, col="oof", sig_mean=sig_mean)
#         df_test.loc[df_test.group.isin([group_i]), "signal"] = signal_test_recovered

#     # re-predict test 
#     for col in TEST_GROUP:
#         df_test.loc[df_test.group == col, "oof"] = ghmm.predict(df_test.loc[df_test.group == col, "signal"].values)
#         df_test.loc[df_test.group == col, viterbi_cols] = ghmm.predict_proba(df_test.loc[df_test.group == col, "signal"].values)

In [ ]:
alpha_cut = 0.8
plt.plot(df_test[df_test.max_typicality > alpha_cut].noise, ".b", alpha=0.1)
# plt.plot(df_test[df_test.max_typicality < alpha_cut].noise, ".r", alpha=0.1)

In [ ]:
df_test.loc[df_test.max_typicality < alpha_cut, "noise"] = np.nan


In [ ]:
df_test["noise"] = df_test["noise"].interpolate()

In [ ]:
fs = 10000
nperseg = 10000
f, t, Zxx = scisig.stft(df_test.noise, fs=fs, nperseg=nperseg)
Zxx[49:52,:] = Zxx[49:52,:] * 0.02
plt.figure()
plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=0.02)
plt.ylim([f[1], f[-1]])
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.yscale('log')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot((df_test["noise"]).values )
plt.plot((df_test["noise"]).rolling(100).mean())

In [ ]:
fig = plt.figure(figsize=(15,5))

fs=10000
fft = np.fft.fft(df_test.noise)
psd = np.abs(fft) ** 2
fftfreq = np.fft.fftfreq(len(psd),1/fs)

i = abs(fftfreq) < 200
plt.grid()
plt.plot(fftfreq[i], 20*np.log10(psd[i]), linewidth=.5)

plt.show()


In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot((df_train["signal_original"] - df_train["signal"]).values)
# plt.plot((df_train["signal_original"] - df_train["signal"]).rolling(100).mean() )

In [ ]:
fig = plt.figure(figsize=(15,5))

fs=10000
fft = np.fft.fft((df_train["signal_original"] - df_train["signal"]).values)
psd = np.abs(fft) ** 2
fftfreq = np.fft.fftfreq(len(psd),1/fs)

i = abs(fftfreq) < 200
plt.grid()
plt.plot(fftfreq[i], 20*np.log10(psd[i]), linewidth=.5)

plt.show()


In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot((df_test["signal_original"] - df_test["signal"]).values )
plt.plot((df_test["signal_original"] - df_test["signal"]).rolling(100).mean())

In [ ]:
fig = plt.figure(figsize=(15,5))

fs=10000
fft = np.fft.fft((df_test["signal_original"] - df_test["signal"]).values)
psd = np.abs(fft) ** 2
fftfreq = np.fft.fftfreq(len(psd),1/fs)

i = abs(fftfreq) < 200
plt.grid()
plt.plot(fftfreq[i], 20*np.log10(psd[i]), linewidth=.5)

plt.show()


In [ ]:
stop

In [ ]:
print(use_cols)

In [ ]:
# scale back
df_train.loc[:,"oof"] = df_train.loc[:,"oof"] + shft
df_test.loc[:,"oof"] = df_test.loc[:,"oof"] + shft
df_train.loc[:,TARGET] = df_train.loc[:,TARGET] + shft
df_test.loc[:,TARGET] = df_test.loc[:,TARGET] + shft

# # save train/test oof and prediction
# np.save('hmm_oof/train_oof_model5.npy', df_train["oof"])
# np.save('hmm_oof/test_oof_model5.npy', df_test["oof"])
# # np.save('hmm_pred/test_pred_model5.npy', df_test[TARGET])

In [ ]:
shft2 = 2
class_num = 11 - shft2
# reduce batch 5&10 open_channels from 11 to 9 (class 2-10) for nn training
df_train.loc[df_train.batch.isin([5,10]) & (df_train.open_channels < shft2), "open_channels"] = shft2
df_train[TARGET] = df_train[TARGET] - shft2

In [ ]:
# cut the signal into sequences
SEQ_LEN = 400
def chop_seq(df_batch_i, is_train):

    df_batch_i_features = []
    df_batch_i_y = []
    df_batch_i_group = []
    
    WHOLE_LEN = 5e5 if is_train else 1e5
    
    for i in range(int(WHOLE_LEN/SEQ_LEN)):

        # (SEQ_LEN, 5)
        tmp = df_batch_i[(SEQ_LEN * i):(SEQ_LEN * (i + 1))]
        df_batch_i_features.append(tmp[use_cols].values)
        df_batch_i_y.append(tmp[TARGET].values)
        df_batch_i_group.append(tmp["group"].values)

    return df_batch_i_features, df_batch_i_y, df_batch_i_group

In [ ]:
# TRAIN
df_train_seq = []
df_train_y = []
groups = []

for batch_i in BATCH_GROUP:
    df_batch_i = df_train[df_train.batch == batch_i]
    df_batch_i_features, df_batch_i_y, df_batch_i_group = chop_seq(df_batch_i, is_train=1)
    df_train_seq.append(df_batch_i_features)
    df_train_y.append(df_batch_i_y)
    groups.append(df_batch_i_group)

df_train_seq = np.array(df_train_seq).reshape(
    [-1, SEQ_LEN, np.array(df_train_seq).shape[-1]])
df_train_y = np.array(df_train_y).reshape([-1, SEQ_LEN])
groups = np.array(groups).reshape([-1, SEQ_LEN])[:,0]

print("TRAIN:", df_train_seq.shape, df_train_y.shape)

In [ ]:
# TEST
df_test_seq = []
df_test_y = []
df_test_groups = []

mini_batch_list = [[2,1], [2,3]]
for batch_i, mini_batch_i in mini_batch_list:
    df_batch_i = df_test[(df_test.batch == batch_i) & (df_test.mini_batch == mini_batch_i)]
    df_batch_i_features, df_batch_i_y, df_test_batch_i_group = chop_seq(df_batch_i, is_train=0)
    df_test_seq.append(df_batch_i_features)
    df_test_y.append(df_batch_i_y)
    df_test_groups.append(df_test_batch_i_group)

df_test_seq = np.array(df_test_seq).reshape(
    [-1, SEQ_LEN, np.array(df_test_seq).shape[-1]])
df_test_y = np.array(df_test_y).reshape([-1, SEQ_LEN])
df_test_groups = np.array(df_test_groups).reshape([-1, SEQ_LEN])[:,0]

print("TEST:", df_test_seq.shape, df_test_y.shape)

In [ ]:
def Classifier(shape_):
    
    def cbr(x, out_layer, kernel, stride, dilation):
        x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
    
    inp = Input(shape = (shape_))
    x = cbr(inp, 64, 5, 1, 1)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = wave_block(x, 64, 3, 8)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = wave_block(x, 64, 3, 4)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = wave_block(x, 128, 3, 1)
    x = cbr(x, 32, 5, 1, 1)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Dense(class_num, activation = 'softmax', name = 'out')(x)
    
    model = models.Model(inputs = inp, outputs = out)
    
    opt = Adam(lr = LR)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
    return model

In [ ]:
K.clear_session()
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)
oof_ = np.zeros([df_train_seq.shape[0], df_train_seq.shape[1], class_num])
preds_ = np.zeros((df_test_seq.shape[0] * df_test_seq.shape[1], class_num))

In [ ]:
df_train_y = pd.get_dummies(df_train_y.reshape([-1])).values.reshape([-1, SEQ_LEN, class_num])
df_test_y = np.zeros([df_train_y.shape[0], df_train_y.shape[1], class_num])

In [ ]:
gkf = GroupKFold(n_splits=5)

for index, (tr_idx, val_idx) in enumerate(gkf.split(df_train_seq, df_train_y, groups)):
    train_x, train_y = df_train_seq[tr_idx], df_train_y[tr_idx]
    valid_x, valid_y = df_train_seq[val_idx], df_train_y[val_idx]
    print("Running folder", index + 1, ": Evaluate on", np.unique(groups[val_idx]))
    print(f'Our training dataset shape is {train_x.shape}')
    print(f'Our validation dataset shape is {valid_x.shape}')
    
    shape_ = (None, train_x.shape[2])
    model = Classifier(shape_)
    cb_lr_schedule = LearningRateScheduler(lr_schedule)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    checkpoint_filepath = 'best_model.h5'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    model.fit(train_x,train_y,
              epochs = nn_epochs,
              callbacks = [cb_lr_schedule, early_stop, model_checkpoint_callback], 
              batch_size = nn_batch_size, verbose = 0,
              validation_data = (valid_x,valid_y))
    model.load_weights(checkpoint_filepath)

    preds_f = model.predict(valid_x)
    f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro') 
    print(f'Training fold {index + 1} completed. macro f1 score : {f1_score_ :1.5f}')
    oof_[val_idx] += preds_f
    te_preds = model.predict(df_test_seq)
    te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
    preds_ += te_preds / gkf.n_splits

In [ ]:
# 0.88612 (class2-10)
print("NN oof F1 score is", f1_score(df_train_y.reshape([-1,class_num]).argmax(axis=1), oof_.reshape([-1,class_num]).argmax(axis=1), average = 'macro').round(5))

In [ ]:
oof_pred = oof_.reshape([-1,class_num]).argmax(axis=1) + shft2
df_train["nn_oof"] = oof_pred
df_train.loc[:,TARGET] = df_train.loc[:,TARGET] + shft2

test_pred = preds_.argmax(axis=1) + shft2
df_test.loc[df_test.group.isin(TEST_GROUP), TARGET] = test_pred
df_test.loc[df_test.oof == 1, TARGET] = 1 # fix class 1 problem
print(f"oof shape is {oof_pred.shape}, test pred shape is {test_pred.shape}")

In [ ]:
# # save oof and prediction
# np.save('hmm_oof/train_nn_oof_model5.npy', oof_pred)
# np.save('hmm_pred/test_nn_oof_model5.npy', df_test[TARGET].values)

In [ ]:
# train target plot
fig = plt.figure(figsize=(12, 6))
plt.grid()
for channel_i in range(11):
    plt.plot(df_train[df_train.open_channels == channel_i].signal_original,
             ".",
             color=color_list[channel_i],
             alpha=0.5)

In [ ]:
# train oof plot
fig = plt.figure(figsize=(12, 6))
plt.grid()
for channel_i in range(11):
    plt.plot(df_train[df_train.nn_oof == channel_i].signal_original,
             ".",
             color=color_list[channel_i],
             alpha=0.5)

In [ ]:
# test oof plot - hmm prediction
fig = plt.figure(figsize=(12, 6))
plt.grid()
for channel_i in range(11):
    plt.plot(df_test[(df_test.oof == channel_i)  & (df_test.group.isin(TEST_GROUP))].signal_original,
             ".",
             color=color_list[channel_i],
             alpha=0.5)
# plt.xlim([0,50000])

In [ ]:
# test predict plot
fig = plt.figure(figsize=(12, 6))
plt.grid()
for channel_i in range(11):
    plt.plot(df_test[(df_test.open_channels == channel_i)  & (df_test.group.isin(TEST_GROUP))].signal_original,
             ".",
             color=color_list[channel_i],
             alpha=0.5)
# plt.xlim([0,50000])